In [ ]:
class Node:
    def __init__(self, nid: int, x: float = 0.0, y: float = 0.0):
        self.id = nid
        self.x = x
        self.y = y

class Edge:
    def __init__(self, u: int, v: int, distance: float):
        self.u = u
        self.v = v
        self.distance = distance